In [27]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [28]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 1
DOLLAR_THRESHOLD = "115009542m"

START_DATE = "20240101"
END_DATE = "20241231"

# Build base name
if SAMPLE_TYPE == "dollar":
    RESAMPLED_NAME = f"{SYMBOL}-{DOLLAR_THRESHOLD}-dollar-{START_DATE}-{END_DATE}"
else:
    RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}-processed.pkl"

In [58]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)

CPU times: user 2.22 ms, sys: 68.5 ms, total: 70.7 ms
Wall time: 71.4 ms


In [59]:
df.shape

(371236, 100)

In [60]:
df.head()

,open,high,low,close,volume,spread,close_delta,close_return,close_log_return,ret_mean_5,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-01-02 01:44:00,1.10381,1.103845,1.103805,1.103835,2.090600e+11,0.000037,0.000020,0.000018,0.000018,0.000016,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 01:45:00,1.10385,1.104010,1.103850,1.103890,3.001600e+11,0.000033,0.000055,0.000050,0.000050,0.000026,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 01:46:00,1.10390,1.103960,1.103885,1.103905,3.997200e+11,0.000034,0.000015,0.000014,0.000014,0.000026,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 01:47:00,1.10391,1.103990,1.103860,1.103900,5.193100e+11,0.000032,-0.000005,-0.000005,-0.000005,0.000024,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025
2024-01-02 01:48:00,1.10389,1.103955,1.103870,1.103900,1.983200e+11,0.000028,0.000000,0.000000,0.000000,0.000015,...,2,1,0.258819,0.965926,0.781831,0.62349,0.394356,0.918958,0.5,0.866025


In [61]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'close_delta',
       'close_return', 'close_log_return', 'ret_mean_5', 'ret_mean_10',
       'ret_mean_15', 'ret_mean_20', 'log_volume', 'rv5', 'log_rv5',
       'sqrt_rv5', 'rv15', 'log_rv15', 'sqrt_rv15', 'rv50', 'log_rv50',
       'sqrt_rv50', 'ema5', 'ema5_slope', 'close_above_ema5', 'ema20',
       'ema20_slope', 'close_above_ema20', 'ema50', 'ema50_slope',
       'close_above_ema50', 'ema100', 'ema100_slope', 'close_above_ema100',
       'atr14', 'atr60', 'atr120', 'log_atr14', 'atr14_percent',
       'atr14_adjusted_return', 'log_atr60', 'atr60_percent',
       'atr60_adjusted_return', 'log_atr120', 'atr120_percent',
       'atr120_adjusted_return', 'adx14', 'plus_di14', 'minus_di14',
       'bb_upper', 'bb_lower', 'bb_mavg', 'bb_width', 'bb_position',
       'dc20_upper', 'dc20_lower', 'dc20_mid', 'dc20_width',
       'close_above_dc20_mid', 'dc20_breakout', 'dc20_breakdown', 'dc50_upper',
       'dc50_lower', 'dc50_mi

## Apply CUSUM Filter

## Calculate CUSUM Threshold

In [116]:
return_std = np.log(df['close']).diff().iloc[1:].std()
return_std * 3

np.float64(0.0003098871099078162)

In [117]:
EVENT_NAME = 'CUSUM'
THRESHOLD = 0.00031
EVENT_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}_{THRESHOLD}.pkl"

In [118]:
EVENT_PATH

PosixPath('../data/events/USDJPY-1m-20240101-20241231_CUSUM_0.00031.pkl')

## Get Events Start Time (t0)

In [119]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def cusum_filter(closes: pd.Series, threshold: float) -> pd.DatetimeIndex:
    # get tEvents using CUSUM filter
    ret = np.log(closes).diff().iloc[1:]
    values = ret.values
    timestamps = ret.index

    t_events_mask = np.zeros_like(values, dtype=bool)

    cum_pos, cum_neg = 0.0, 0.0

    for i in tqdm(range(len(values))):
        cum_pos = max(0.0, cum_pos + values[i])
        cum_neg = min(0.0, cum_neg + values[i])

        if cum_pos > threshold:
            t_events_mask[i] = True
            cum_pos = 0.0
        elif cum_neg < -threshold:
            t_events_mask[i] = True
            cum_neg = 0.0

    return timestamps[t_events_mask]


In [120]:
%%time

t_events = cusum_filter(df['close'], threshold=THRESHOLD)

100%|█████████████████████████████████████| 371235/371235 [00:00<00:00, 3144057.45it/s]

CPU times: user 125 ms, sys: 6.61 ms, total: 132 ms
Wall time: 128 ms


In [121]:
t_events

DatetimeIndex(['2024-01-02 01:57:00', '2024-01-02 02:12:00',
               '2024-01-02 02:22:00', '2024-01-02 02:40:00',
               '2024-01-02 02:53:00', '2024-01-02 03:09:00',
               '2024-01-02 03:45:00', '2024-01-02 04:10:00',
               '2024-01-02 04:42:00', '2024-01-02 04:58:00',
               ...
               '2024-12-30 20:31:00', '2024-12-30 20:57:00',
               '2024-12-30 21:03:00', '2024-12-30 21:43:00',
               '2024-12-30 21:58:00', '2024-12-30 22:05:00',
               '2024-12-30 22:52:00', '2024-12-30 23:00:00',
               '2024-12-30 23:15:00', '2024-12-30 23:36:00'],
              dtype='datetime64[ns]', name='timestamp', length=29678, freq=None)

In [122]:
EVENT_PATH

PosixPath('../data/events/USDJPY-1m-20240101-20241231_CUSUM_0.00031.pkl')

In [123]:
t_events.to_series().to_pickle(EVENT_PATH)